# day 10

https://adventofcode.com/2019/day/10

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day10.txt')

LOGGER = logging.getLogger('day10')

## part 1

### problem statement:

#### loading data

In [ ]:
test_1 = """.#..#
.....
#####
....#
...##"""

test_2 = """......#.#.
#..#.#....
..#######.
.#.#.###..
.#..#.....
..#....#.#
#..#....#.
.##.#..###
##...#..#.
.#....####"""

test_3 = """#.#...#.#.
.###....#.
.#....#...
##.#.#.#.#
....#.#.#.
.##..###.#
..#...##..
..##....##
......#...
.####.###."""

test_4 = """.#..#..###
####.###.#
....###.#.
..###.##.#
##.##.#.#.
....###..#
..#.#..#.#
#..#.#.###
.##...##.#
.....#.#.."""

test_5 = """.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##"""

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read()

#### function def

In [ ]:
import math

In [ ]:
ASTEROID = '#'
EMPTY = '.'

from collections import defaultdict


def dist(p, q):
    return math.sqrt(sum((px - qx) ** 2.0 for px, qx in zip(p, q)))


class AsteroidField:
    def __init__(self, s):
        self.s = s.strip()
        self.asteroids = {(int(j), int(i)): val == ASTEROID
                          for (i, row) in enumerate(self.s.split('\n'))
                          for (j, val) in enumerate(row)}
        self.asteroid_locations = [
            k for (k, v) in self.asteroids.items() if v]
    
    def _theta_dict(self, loc):
        theta_dict = defaultdict(dict)
        x0, y0 = loc
        for al in self.asteroid_locations:
            if al != loc:
                x1, y1 = al
                rise = y1 - y0
                run = x1 - x0
                theta = math.atan2(rise, run)
                d = dist(loc, al)
                theta_dict[theta][d] = al
        return theta_dict
    
    def num_seen_from(self, loc):
        return len(self._theta_dict(loc))
    
    def best_monitoring_loc(self):
        best_num = 0
        best_loc = None
        for al in self.asteroid_locations:
            num_here = self.num_seen_from(al)
            if num_here > best_num:
                best_loc = al
                best_num = num_here
        return (best_loc, best_num)
    
    def blasted_by_lasers(self, loc):
        theta_dict = self._theta_dict(loc)
        last_angle = -math.pi / 2 - 1e-6
        num_blasted = 0
        while theta_dict:
            # point it to the next asteroid
            try:
                ray_angle = min(k for k in theta_dict if k > last_angle)
            except ValueError:
                # start the loop over again
                last_angle = -math.pi
                continue
            candidates = theta_dict[ray_angle]
            shortest_dist = min(candidates.keys())
            nearest_loc = candidates.pop(shortest_dist)
            num_blasted += 1
            LOGGER.debug(f"blasting asteroid {num_blasted} at {(nearest_loc)}")
            yield (nearest_loc, shortest_dist)
            if not candidates:
                del theta_dict[ray_angle]
            last_angle = ray_angle

In [ ]:
def q_1(data):
    af = AsteroidField(data)
    return af.best_monitoring_loc()

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    af = AsteroidField(test_1)
    assert af.num_seen_from((1, 0)) == 7
    assert af.num_seen_from((3, 4)) == 8

    assert q_1(test_1) == ((3, 4), 8)
    assert q_1(test_2) == ((5, 8), 33)
    assert q_1(test_3) == ((1, 2), 35)
    assert q_1(test_4) == ((6, 3), 41)
    assert q_1(test_5) == ((11, 13), 210)
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
def q_2(data):
    af = AsteroidField(data)
    best_loc, num_thetas = af.best_monitoring_loc()
    for (i, (blasted_asteroid_loc, d)) in enumerate(af.blasted_by_lasers(best_loc)):
        if i == 199:
            return blasted_asteroid_loc

#### tests

In [ ]:
test_6 = """.#....#####...#..
##...##.#####..##
##...#...#.#####.
..#.....#...###..
..#.#.....#....##"""

def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    af = AsteroidField(test_6)
    assert af.best_monitoring_loc() == ((8, 3), 30)
    assert q_2(test_5) == (8, 2)
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
x, y = q_2(load_data())

100 * x + y

fin